

**Microdados ENEM 2023**



**Data Cleaning**



*   Neste notebook, realizarei a limpeza dos microdados do ENEM 2022.
*   Os microdados se constituem no menor nível de desagregação de dados recolhidos por pesquisas, avaliações e exames realizados. No caso do ENEM, os dados estão por participante.
* A limpeza de dados é necessária para performar a análise exploratória de dados. Dado o alto volume de dados e a sua origem (dados reais), algumas tarefas devem ser realizadas:

  * Identificação e tratamento de valores nulos e duplicados, de acordo com os objetivos da análise.
  * Remoção de variáveis irrelevantes para a análise.
  * Feature engineering: Criação e alteração de variáveis existentes. Aqui, irei fundir, remover e renomear categorias com base na melhor formatação para o meu objetivo. Além disso, converter colunas para o tipo de dado correto também será importante.
  * Otimização de memória: Conversão de variáveis a tipos de dados menores, a fim de melhorar a performance, possibilitando a leitura e manipulação dos dados em menor tempo, sem que haja a perda de informação.


* Irei efetuar duas análises após a limpeza. O objetivo de cada uma delas guiará
decisões tomadas futuramente neste notebook.

* Na Análise de Desempenho, tenho como foco analisar o perfil de candidatos que obtêm determinadas notas, quais variáveis se relacionam com as notas e como estas poderiam ser utilizadas para predição. Portanto, é importante utilizar apenas dados de estudantes que estavam presentes em ambos os dias de prova, que de fato obtiveram um resultado. Incluir todos os alunos introduziria assimetrias e distorções na análise.

* Na Análise de Abstenção, tenho como foco analisar quais os fatores que influenciam a ausência do candidato. Portanto, é necessário incluir todos os alunos.

**Importando as Bibliotecas**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os

**2. Coletando os dados**
* Considerando o alto volume de dados, irei ler o dataset microdados em "chunks", partes menores, unindo tudo ao fim. Isso tornará a leitura mais eficiente.

In [4]:
chunk_size = 50_000
chunks = []
microdados_path = "D:\DataScience\Dataset\MICRODADOS_ENEM_2023.csv"
for chunk in pd.read_csv(microdados_path, sep=';', encoding='ISO-8859-1', chunksize=chunk_size):
   chunks.append(chunk)

microdados = pd.concat(chunks, ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\DataScience\\Dataset\\MICRODADOS_ENEM_2023.csv'

**3. Entendimento inicial dos dados e limpeza dos dados**
* Nesta etapa irei observar superficialmente os dados, obtendo dimensões, tipos de dados das variáveis, valores nulos e duplicados, estatísticas descritivas, entre outros.
* Será realizada a limpeza deles também. Irei remover colunas desnecessárias, converter variáveis para os tipos de dado corretos, reduzir o tamanho do dataset, tratar valores nulos e outliers, entre outras tarefas.

**3.2 Informações gerais sobre os dados**

In [ ]:
microdados.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210059085136,2023,14,M,2,1,1,1,17,1,...,C,C,B,B,A,B,B,A,A,B
1,210059527735,2023,12,M,2,1,0,1,16,1,...,B,A,B,B,A,A,C,A,D,B
2,210061103945,2023,6,F,1,1,1,1,0,1,...,B,A,A,B,A,A,A,A,A,B
3,210060214087,2023,2,F,1,3,1,2,0,2,...,A,A,A,B,A,A,D,A,A,B
4,210059980948,2023,3,F,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,A,A


In [ ]:
microdados.shape

(3933955, 76)

In [ ]:
microdados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 76 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   NU_INSCRICAO            int64  
 1   NU_ANO                  int64  
 2   TP_FAIXA_ETARIA         int64  
 3   TP_SEXO                 object 
 4   TP_ESTADO_CIVIL         int64  
 5   TP_COR_RACA             int64  
 6   TP_NACIONALIDADE        int64  
 7   TP_ST_CONCLUSAO         int64  
 8   TP_ANO_CONCLUIU         int64  
 9   TP_ESCOLA               int64  
 10  TP_ENSINO               float64
 11  IN_TREINEIRO            int64  
 12  CO_MUNICIPIO_ESC        float64
 13  NO_MUNICIPIO_ESC        object 
 14  CO_UF_ESC               float64
 15  SG_UF_ESC               object 
 16  TP_DEPENDENCIA_ADM_ESC  float64
 17  TP_LOCALIZACAO_ESC      float64
 18  TP_SIT_FUNC_ESC         float64
 19  CO_MUNICIPIO_PROVA      int64  
 20  NO_MUNICIPIO_PROVA      object 
 21  CO_UF_PROVA             int64  

In [ ]:
microdados.describe()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,...,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q005
count,3.933955e+06,3933955.0,3.933955e+06,3.933955e+06,3.933955e+06,3.933955e+06,3.933955e+06,3.933955e+06,3.933955e+06,1.339081e+06,...,2.692427e+06,3.933955e+06,2.822643e+06,2.822643e+06,2.822643e+06,2.822643e+06,2.822643e+06,2.822643e+06,2.822643e+06,3.933955e+06
mean,2.100601e+11,2023.0,5.096262e+00,1.043938e+00,2.058200e+00,1.028277e+00,1.684697e+00,2.445029e+00,1.415810e+00,1.005142e+00,...,5.338357e+02,4.570085e-01,1.152402e+00,1.215452e+02,1.392952e+02,1.185090e+02,1.297970e+02,1.086520e+02,6.177985e+02,3.688928e+00
std,1.154550e+06,0.0,3.871855e+00,4.120029e-01,1.003314e+00,2.127761e-01,7.467924e-01,4.450337e+00,6.018231e-01,7.152547e-02,...,1.316478e+02,4.981484e-01,8.001447e-01,3.565444e+01,5.161563e+01,4.340828e+01,4.377326e+01,6.160248e+01,2.146211e+02,1.409879e+00
min,2.100580e+11,2023.0,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,2.100591e+11,2023.0,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,...,4.312000e+02,0.000000e+00,1.000000e+00,1.200000e+02,1.200000e+02,1.000000e+02,1.200000e+02,6.000000e+01,5.000000e+02,3.000000e+00
50%,2.100601e+11,2023.0,3.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,...,5.236000e+02,0.000000e+00,1.000000e+00,1.200000e+02,1.400000e+02,1.200000e+02,1.200000e+02,1.200000e+02,6.200000e+02,4.000000e+00
75%,2.100611e+11,2023.0,7.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00,1.000000e+00,...,6.301000e+02,1.000000e+00,1.000000e+00,1.400000e+02,1.800000e+02,1.400000e+02,1.600000e+02,1.600000e+02,7.800000e+02,4.000000e+00
max,2.100621e+11,2023.0,2.000000e+01,4.000000e+00,5.000000e+00,4.000000e+00,4.000000e+00,1.700000e+01,3.000000e+00,2.000000e+00,...,9.586000e+02,1.000000e+00,9.000000e+00,2.000000e+02,2.000000e+02,2.000000e+02,2.000000e+02,2.000000e+02,1.000000e+03,2.000000e+01


**3.3 Valores nulos e duplicados**

In [ ]:
microdados['NU_INSCRICAO'].duplicated().sum()
microdados.duplicated().sum()

np.int64(0)

In [ ]:
null_df = (microdados.isna().sum() / len(microdados) * 100).to_frame().rename(columns={0: 'null_pct'})
null_df['null_count'] = microdados.isna().sum()
null_df.sort_values(by=['null_pct'], ascending=False).head(35)

,null_pct,null_count
SG_UF_ESC,75.635054,2975449
CO_UF_ESC,75.635054,2975449
NO_MUNICIPIO_ESC,75.635054,2975449
CO_MUNICIPIO_ESC,75.635054,2975449
TP_LOCALIZACAO_ESC,75.635054,2975449
TP_SIT_FUNC_ESC,75.635054,2975449
TP_DEPENDENCIA_ADM_ESC,75.635054,2975449
TP_ENSINO,65.960948,2594874
CO_PROVA_CN,31.559283,1241528
TX_GABARITO_MT,31.559283,1241528


* Não há observações duplicadas no dataset.

* Variáveis referentes à escola possuem um alto percentual de nulos, acima de 80%, e portanto deverão ser removidas.

* É possível ver um padrão nas variáveis referentes à nota e código de prova, possuindo o mesmo percentual de nulos para cada dia de aplicação (26% para as provas de ciências da natureza e matemática e 23.9% para as provas de ciências humanas, linguagens e suas tecnologias e redação).
* Provavelmente esses valores nulos representam alunos que não compareceram ou que foram eliminados, vamos investigar adiante.

In [ ]:
# Busca valores nulos nos dias em que o aluno não compareceu no segundo dia de prova.
segundo_dia = ['CO_PROVA_CN',
 'NU_NOTA_CN',
 'TX_RESPOSTAS_CN',
 'TX_GABARITO_CN',
 'CO_PROVA_MT',
 'NU_NOTA_MT',
 'TX_RESPOSTAS_MT',
 'TX_GABARITO_MT']
microdados[segundo_dia].loc[(microdados['TP_PRESENCA_MT'] == 0) & (microdados['TP_PRESENCA_CN'] == 0)].isna().sum()

CO_PROVA_CN        1239316
NU_NOTA_CN         1239316
TX_RESPOSTAS_CN    1239316
TX_GABARITO_CN     1239316
CO_PROVA_MT        1239316
NU_NOTA_MT         1239316
TX_RESPOSTAS_MT    1239316
TX_GABARITO_MT     1239316
dtype: int64

In [ ]:
# Busca por valores nulos nos dias em que o aluno foi eliminado para o segundo dia de prova.
microdados[segundo_dia].loc[(microdados['TP_PRESENCA_MT'] == 2) & (microdados['TP_PRESENCA_CN'] == 2)].isna().sum()

CO_PROVA_CN        2212
NU_NOTA_CN         2212
TX_RESPOSTAS_CN    2212
TX_GABARITO_CN     2212
CO_PROVA_MT        2212
NU_NOTA_MT         2212
TX_RESPOSTAS_MT    2212
TX_GABARITO_MT     2212
dtype: int64

In [ ]:
# Busca por valores nulos nos dias em que o aluno não compareceu no primeiro dia de prova.
primeiro_dia = ['CO_PROVA_CH', 'CO_PROVA_LC', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_GABARITO_CH',
       'TX_GABARITO_LC', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
       'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO']
microdados[primeiro_dia].loc[(microdados['TP_PRESENCA_CH'] == 0) & (microdados['TP_PRESENCA_LC'] == 0)].isna().sum()

CO_PROVA_CH          1106714
CO_PROVA_LC          1106714
NU_NOTA_CH           1106714
NU_NOTA_LC           1106714
TX_RESPOSTAS_CH      1106714
TX_RESPOSTAS_LC      1106714
TX_GABARITO_CH       1106714
TX_GABARITO_LC       1106714
TP_STATUS_REDACAO    1106714
NU_NOTA_COMP1        1106714
NU_NOTA_COMP2        1106714
NU_NOTA_COMP3        1106714
NU_NOTA_COMP4        1106714
NU_NOTA_COMP5        1106714
NU_NOTA_REDACAO      1106714
dtype: int64

In [ ]:
# Busca por valores nulos nos dias em que o aluno foi eliminado no primeiro dia de prova.
microdados[primeiro_dia].loc[(microdados['TP_PRESENCA_CH'] == 2) & (microdados['TP_PRESENCA_LC'] == 2)].isna().sum()

CO_PROVA_CH          4598
CO_PROVA_LC          4598
NU_NOTA_CH           4598
NU_NOTA_LC           4598
TX_RESPOSTAS_CH      4598
TX_RESPOSTAS_LC      4598
TX_GABARITO_CH       4598
TX_GABARITO_LC       4598
TP_STATUS_REDACAO    4598
NU_NOTA_COMP1        4598
NU_NOTA_COMP2        4598
NU_NOTA_COMP3        4598
NU_NOTA_COMP4        4598
NU_NOTA_COMP5        4598
NU_NOTA_REDACAO      4598
dtype: int64

In [ ]:
microdados['TP_PRESENCA_LC'].value_counts(normalize=True) * 100

TP_PRESENCA_LC
1    71.75077
0    28.13235
2     0.11688
Name: proportion, dtype: float64

In [ ]:

microdados['TP_PRESENCA_MT'].value_counts(normalize=True) * 100

TP_PRESENCA_MT
1    68.440717
0    31.503055
2     0.056228
Name: proportion, dtype: float64

* É possível perceber que todas as observações que contêm valores nulos em variáveis contendo informações sobre as provas ocorrem por não comparecimento ou eliminação do estudante.

* Considerando que variáveis contendo informações sobre o status da redacao, código da prova, gabaritos e respostas serão removidas porque não são relevantes para a análise, não imputarei valores nulos nelas.

* Entre os principais objetivos da Análise de Desempenho, estão, entender o perfil dos candidatos que tiram determinadas notas, quais variáveis se relacionam com as notas e como estas podem ser utilizadas para predição.
* Portanto, candidatos faltantes que possuem notas nulas serão separados futuramente. Por enquanto, imputarei essas observações com nota zero. Isso será útil para analisar distribuições e fatores que influenciam o comparecimento (ou não) em cada dia de prova, compondo a Análise de Abstenção.

* Para candidatos que foram eliminados, os quais também possuem nota nula nas provas, irei dropá-los de antemão, pois representam uma parcela muito pequena da população e não trazem informação relevante.

In [ ]:
clean_microdados = microdados.copy()

In [ ]:
# Eliminar registros eliminados.
eliminated_records = clean_microdados.loc[(clean_microdados['TP_PRESENCA_CN'] == 2)
                                          | (clean_microdados['TP_PRESENCA_CH'] == 2)
                                          | (microdados['TP_PRESENCA_LC'] == 2)
                                          | (microdados['TP_PRESENCA_MT'] == 2)].index
clean_microdados = clean_microdados.drop(eliminated_records)

# Imputar nota zero para quem não fez o respectivo dia de exame.
to_impute_grade = ['NU_NOTA_CN', 'NU_NOTA_MT', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']
clean_microdados[to_impute_grade] = clean_microdados[to_impute_grade].fillna(0)

**3.4 Removendo colunas desnecessárias**

 É possível perceber que temos um grande número de colunas. Observando o dicionário de variáveis, muitas dessas não nos interessam, e portanto, serão removidas. Entram nesse grupo:

* Variáveis informando códigos (códigos de prova para cada área do conhecimento, de cidades, unidades federativas, entre outros).

* Variáveis que podem introduzir viés na análise, como as que identificam cor e raça.

* Variáveis com gabaritos e respostas de questões.

* Variáveis altamente desbalanceadas e que podem ser refletidas em valores de outras colunas, como o status da redação.
Variáveis informando nacionalidade, número de inscrição e ano de conclusão do ensino médio.

* Variáveis com um identificador único para cada estudante.

* Variáveis contendo respostas para perguntas do questionário que não trazem valor para a análise ou que contêm múltiplas possibilidades para uma categoria.

* Variáveis com alto percentual de valores nulos, como visto acima.

In [ ]:
# Definindo colunas para serem descartadas.
to_drop = ['NU_INSCRICAO', 'NU_ANO', 'TP_NACIONALIDADE', 'TP_COR_RACA', 'TP_ANO_CONCLUIU',
           'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC',
           'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'TP_ENSINO', 'TP_STATUS_REDACAO', 'CO_MUNICIPIO_PROVA', 'CO_UF_PROVA',
           'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH',
           'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
           'TX_GABARITO_MT', 'Q003', 'Q004', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
           'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q023']
clean_microdados = clean_microdados.drop(columns=to_drop)
clean_microdados.shape

(3927146, 30)

* Foi reduzido de 76 colunas para 30!

**3.5 Alterando variáveis**

* De forma geral, variáveis de natureza categórica estão com tipos numéricos. Irei convertê-las com os respectivos valores categóricos a fim de tornar a análise interpretável e de fácil entendimento. Ao mesmo tempo em que converto, irei unir/dropar categorias, de acordo com o melhor critério.

* Irei também padronizar os valores e renomear colunas para melhor entendimento.

In [ ]:
# Standardaze apresenta nomes, formatando-os e renomeando-os.
clean_microdados.columns = [x.lower() for x in clean_microdados.columns]

prefixes_to_remove = ['tp_', 'in_', 'sg_', 'nu_', 'no_']
clean_microdados.columns = clean_microdados.columns.to_series().replace(to_replace='^(' + '|'.join(prefixes_to_remove) + ')', value='', regex=True)

to_rename = {'q001': 'escolaridade_pai',
             'q002': 'escolaridade_mae',
             'q003': 'ocupacao_pai',
             'q004': 'ocupacao_mae',
             'q005': 'numero_pessoas_em_casa',
             'q006': 'renda_familiar_mensal',
             'q022': 'possui_celular_em_casa',
             'q024': 'possui_computador_em_casa',
             'q025': 'acesso_internet_em_casa',
             'st_conclusao': 'status_conclusao_ensino_medio'}

clean_microdados = clean_microdados.rename(columns=to_rename)

# Substitua valores numéricos por valores categóricos em recursos categóricos no tipo de dados numéricos.
# Mesclar categorias de idade semelhantes em uma e atribuir-lhes um valor categórico para melhor interpretação da análise.
faixa_etaria_mapping = {
    'Adolescente (< 18)': [1, 2],                   # < 18
    'Jovem adulto (18-24)': [3, 4, 5, 6, 7, 8, 9],  # 18-24
    'Adulto jovem (25-35)': [10, 11, 12],           # 25-35
    'Adulto de meia idade (36-45)': [13, 14],       # 36-45
    'Meia idade (46-55)': [15, 16],                 # 46-55
    'Pré aposentadoria (56-65)': [17, 18],          # 56-65
    'Idoso (> 66)': [19, 20]                        # > 66
}

replaced_faixa_etaria = dict()

for group, keys in faixa_etaria_mapping.items():
    for key in keys:
        replaced_faixa_etaria[key] = group

clean_microdados['faixa_etaria'] = clean_microdados['faixa_etaria'].replace(to_replace=replaced_faixa_etaria)

# Substituir estado civil
estado_civil_mapping = {
    0: 'Não informado',
    1: 'Solteiro(a)',
    2: 'Casado(a)/União Estável',
    3: 'Divorciado(a)/Separado(a)',
    4: 'Viúvo(a)'
}
clean_microdados['estado_civil'] = clean_microdados['estado_civil'].replace(estado_civil_mapping)

# Substituir situação de conclusão do ensino médio.
st_conclusao_mapping = {
    1: 'Concluído',
    2: 'Último ano',
    3: 'Cursando',
    4: 'Não concluído'
}
clean_microdados['status_conclusao_ensino_medio'] = clean_microdados['status_conclusao_ensino_medio'].replace(st_conclusao_mapping)

# Substituir tipo da escola.
escola_mapping = {
    1: 'Não respondeu',
    2: 'Pública',
    3: 'Privada'
}
clean_microdados['escola'] = clean_microdados['escola'].replace(escola_mapping)

# Substitua presença e língua.
clean_microdados[['presenca_cn', 'presenca_ch', 'presenca_lc', 'presenca_mt']] = clean_microdados[['presenca_cn', 'presenca_ch', 'presenca_lc', 'presenca_mt']].replace({1: 'Presente', 0: 'Ausente'})
clean_microdados['lingua'] = clean_microdados['lingua'].replace({0: 'Inglês', 1: 'Espanhol'})

# Substitua as respostas das perguntas.
escolaridade_mapping = {
    'A': 'Nunca estudou',
    'B': 'Ensino fundamental incompleto',
    'C': 'Ensino fundamental incompleto',
    'D': 'Ensino fundamental completo',
    'E': 'Ensino médio completo',
    'F': 'Ensino superior completo',
    'G': 'Pós-graduação',
    'H': 'Não sei'
}
clean_microdados['escolaridade_pai'] = clean_microdados['escolaridade_pai'].replace(escolaridade_mapping)
clean_microdados['escolaridade_mae'] = clean_microdados['escolaridade_mae'].replace(escolaridade_mapping)

# Substitua a renda.
renda_mapping = {
    'A': 'Nenhuma Renda',
    'B': 'Até R$ 1.212,00',
    'C': 'R$ 1.212,01 - R$ 1.818,00',
    'D': 'R$ 1.818,01 - R$ 3.030,00',
    'E': 'R$ 1.818,01 - R$ 3.030,00',
    'F': 'R$ 3.030,01 - R$ 4.848,00',
    'G': 'R$ 3.030,01 - R$ 4.848,00',
    'H': 'R$ 4.848,01 - R$ 7.272,00',
    'I': 'R$ 4.848,01 - R$ 7.272,00',
    'J': 'R$ 7.272,01 - R$ 10.908,00',
    'K': 'R$ 7.272,01 - R$ 10.908,00',
    'L': 'R$ 7.272,01 - R$ 10.908,00',
    'M': 'R$ 10.908,01 - R$ 18.180,00',
    'N': 'R$ 10.908,01 - R$ 18.180,00',
    'O': 'R$ 10.908,01 - R$ 18.180,00',
    'P': 'R$ 18.180,01 - R$ 24.240,00',
    'Q': 'Acima de R$ 24.240,00'
}
clean_microdados['renda_familiar_mensal'] = clean_microdados['renda_familiar_mensal'].replace(renda_mapping)

# Substitua para ver a quantidad de celulares/computador por indivíduo.
computador_celular_mapping = {
    'A': 'Não',
    'B': 'Um',
    'C': 'Dois ou mais',
    'D': 'Dois ou mais',
    'E': 'Dois ou mais'
}
clean_microdados['possui_celular_em_casa'] = clean_microdados['possui_celular_em_casa'].replace(computador_celular_mapping)
clean_microdados['possui_computador_em_casa'] = clean_microdados['possui_computador_em_casa'].replace(computador_celular_mapping)

internet_mapping = {
    'A': 'Não',
    'B': 'Sim'
}
clean_microdados['acesso_internet_em_casa'] = clean_microdados['acesso_internet_em_casa'].replace(internet_mapping)

treineiro_mapping = {
    0: 'Não',
    1: 'Sim'
}
clean_microdados['treineiro'] = clean_microdados['treineiro'].replace(treineiro_mapping)

**3.6 Otimização de memória**

* Irei converter os tipos das variáveis para tipos que consomem menos memória sem perder informação, a fim de otimizar a análise e manipulação dos dados.

* Após isso irei salvar o dataset resultante em um formato parquet, uma vez que o csv não mantém os tipos convertidos.

* Uma vez que as notas vão de 0 a 1000 e têm precisão de 1 casa decimal, converterei-as de float64 para float32. Não será utilizado float16 pois o pyarrow não suporta esse tipo e não será possível salvar em parquet.

* A única variável int é número_pessoas_em_casa. Esta será convertida de int64 para int8, sem perder informação, uma vez que ela guarda valores de 1 a 20. Através do tipo int8 podemos representar números de -128 a 127.

* Variáveis com um número limitado de categorias exclusivas serão convertidas para category.

In [ ]:
# Obtain and convert columns to lower memory consumption data types.
to_float32 = clean_microdados.select_dtypes('float64').columns.tolist()
to_int8 = 'numero_pessoas_em_casa'
to_category = clean_microdados.select_dtypes('object').columns.tolist()
to_category.remove('municipio_prova')

clean_df = clean_microdados.copy()
clean_df[to_float32] = clean_df[to_float32].astype('float32')
clean_df[to_category] = clean_df[to_category].astype('category')


In [ ]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3927146 entries, 0 to 3933954
Data columns (total 30 columns):
 #   Column                         Dtype   
---  ------                         -----   
 0   faixa_etaria                   category
 1   sexo                           category
 2   estado_civil                   category
 3   status_conclusao_ensino_medio  category
 4   escola                         category
 5   treineiro                      category
 6   municipio_prova                object  
 7   uf_prova                       category
 8   presenca_cn                    category
 9   presenca_ch                    category
 10  presenca_lc                    category
 11  presenca_mt                    category
 12  nota_cn                        float32 
 13  nota_ch                        float32 
 14  nota_lc                        float32 
 15  nota_mt                        float32 
 16  lingua                         category
 17  nota_comp1                     f

In [ ]:
clean_df.sample(5)

,faixa_etaria,sexo,estado_civil,status_conclusao_ensino_medio,escola,treineiro,municipio_prova,uf_prova,presenca_cn,presenca_ch,...,nota_comp4,nota_comp5,nota_redacao,escolaridade_pai,escolaridade_mae,numero_pessoas_em_casa,renda_familiar_mensal,possui_celular_em_casa,possui_computador_em_casa,acesso_internet_em_casa
2414771,Jovem adulto (18-24),F,Solteiro(a),Último ano,Privada,Não,Fortaleza,CE,Presente,Presente,...,200.0,180.0,920.0,Ensino superior completo,Ensino superior completo,5,"R$ 18.180,01 - R$ 24.240,00",Dois ou mais,Dois ou mais,Sim
1863161,Jovem adulto (18-24),M,Solteiro(a),Último ano,Pública,Não,João Pessoa,PB,Presente,Presente,...,180.0,200.0,880.0,Ensino fundamental incompleto,Ensino fundamental incompleto,3,"R$ 1.818,01 - R$ 3.030,00",Dois ou mais,Um,Sim
3047083,Jovem adulto (18-24),F,Solteiro(a),Último ano,Pública,Não,Picuí,PB,Presente,Presente,...,0.0,0.0,0.0,Ensino fundamental incompleto,Ensino fundamental incompleto,1,Nenhuma Renda,Um,Não,Sim
602989,Adulto jovem (25-35),M,Casado(a)/União Estável,Último ano,Pública,Não,Duque de Caxias,RJ,Ausente,Ausente,...,0.0,0.0,0.0,Não sei,Ensino fundamental incompleto,4,"Até R$ 1.212,00",Dois ou mais,Um,Sim
1608726,Adulto de meia idade (36-45),F,Divorciado(a)/Separado(a),Concluído,Não respondeu,Não,Jaboatão dos Guararapes,PE,Presente,Presente,...,0.0,0.0,0.0,Ensino fundamental incompleto,Ensino fundamental incompleto,2,Nenhuma Renda,Um,Não,Sim


In [ ]:
# Convert the memory optimized data to a parquet file in order to maintain the converted data types.
path = 'D:\DataScience\Dataset\clean_df.parquet'
clean_df.to_parquet(path, index=False)

In [ ]:
# Read the memory optimized data.
path = 'D:\DataScience\Dataset\clean_df.parquet'
df = pd.read_parquet(path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3927146 entries, 0 to 3927145
Data columns (total 30 columns):
 #   Column                         Dtype   
---  ------                         -----   
 0   faixa_etaria                   category
 1   sexo                           category
 2   estado_civil                   category
 3   status_conclusao_ensino_medio  category
 4   escola                         category
 5   treineiro                      category
 6   municipio_prova                object  
 7   uf_prova                       category
 8   presenca_cn                    category
 9   presenca_ch                    category
 10  presenca_lc                    category
 11  presenca_mt                    category
 12  nota_cn                        float32 
 13  nota_ch                        float32 
 14  nota_lc                        float32 
 15  nota_mt                        float32 
 16  lingua                         category
 17  nota_comp1                 

**4. Conclusão**

* Excelente! Através da limpeza dos dados foi possível reduzir o tamanho do dataset de +2 GB para +767.2 MB. Agora poderemos realizar a análise e manipulação dos dados de forma eficiente.
* Tarefas realizadas:
  * Identificação e tratamento de valores nulos e duplicados, de acordo com os objetivos da análise.
  * Remoção de variáveis irrelevantes para a análise.
  * Feature engineering: Criação e alteração de variáveis existentes. Aqui, irei fundir, remover e renomear categorias com base na melhor formatação para o meu objetivo. Além disso, converter colunas para o tipo de dado correto também será importante.
  * Otimização de memória: Conversão de variáveis a tipos de dados menores, a fim de melhorar a performance, possibilitando a leitura e manipulação dos dados em menor tempo, sem que haja a perda de informação.